In [1]:
import pandas as pd
import re

In [2]:
def map_code_values(category_code_list, print_errors=False):
    equal_split = re.compile(r'\s?=\s?')
    cont_num_range = re.compile(r'\d+\s?-\s?\d+')
    code_map = {}
    error_map = {}
    i = 0 # col index
    for code_list in category_code_list:
        # check codes to follow "num = category pattern"
        if 'no code' in code_list:
            code_list = ['no code = continuous numerical value']
        if bool(cont_num_range.search(code_list[0])) == True:
            # if numerical range is first ie "1 - 10"
            code_list[0] = code_list[0] + ' = continuous numerical range'
        elif bool(cont_num_range.search(code_list[-1])) == True:
            # if numerical range is last ie "1946 - 2012"
            code_list[-1] = code_list[-1] + ' = continuous numerical range'
        elif 'continued' in code_list[-1]:
            # if (continued in next cell) is last, delete from list
            del code_list[-1]

        # convert codes to dict
        try:
            code_map[i] = dict([re.split(equal_split.pattern, code.replace("'", "").strip()) for code in code_list])
        except ValueError as e:
            error_map[i] = (code_list, e)
        
        # increment col index
        i += 1
    if print_errors == True:
        for k,v in error_map.items():
            print(f'codelist: {k} | error: {v}\n-----------')
    return code_map

In [8]:
codebook_col_map = {
    'CATEOGORIES': 'category', 
    'File order': 'file_order', 
    'Variable\r\nname': 'col_name', 
    'Variable type': 'col_type',
    'Len-\r\ngth': 'length', 
    'Format': 'format', 
    'Label': 'col_description', 
    'Values/Format codes': 'codes', 
    'Unnamed: 8': 'empty_col' # (continued in next cell) code prompt?
}
df_codebook = pd.read_csv('031022-codebook.csv').rename(columns=codebook_col_map).drop(['empty_col'], axis=1)
df_codebook.head(5)

,category,file_order,col_name,col_type,length,format,col_description,codes
0,0,1.0,PUBID,Char,5.0,NaN,Building identifier,00001 - 06720
1,0,2.0,REGION,Char,1.0,$REGION.,Census region,'1' = 'Northeast'\r\n'2' = 'Midwest'\r\n'3' = ...
2,0,3.0,CENDIV,Char,1.0,$CENDIV.,Census division,'1' = 'New England'\r\n'2' = 'Middle Atlantic'...
3,0,4.0,PBA,Char,2.0,$PBA.,Principal building activity,'01' = 'Vacant'\r\n'02' = 'Office'\r\n'04' = '...
4,1,5.0,FREESTN,Char,1.0,$YES.,Freestanding building,1' = 'Yes'\r\nMissing='No'


In [9]:
categories = df_codebook.category.unique()
print(f'categories: {categories}')
category_map = {}
category_dfs = {}
for cat_name in categories:
    if cat_name != 'get rid of':
        category_map[cat_name] = df_codebook[df_codebook['category'] == cat_name].col_name.values
        category_dfs[cat_name] = df_codebook[df_codebook['category'] == cat_name]
print(f"category = 1 columns: {category_map['0']}")
category_dfs['0']

categories: ['0' '1' '2' '3' '4' '5' '6' '7' 'get rid of' '10']
category = 1 columns: ['PUBID' 'REGION' 'CENDIV' 'PBA']


,category,file_order,col_name,col_type,length,format,col_description,codes
0,0,1.0,PUBID,Char,5.0,NaN,Building identifier,00001 - 06720
1,0,2.0,REGION,Char,1.0,$REGION.,Census region,'1' = 'Northeast'\r\n'2' = 'Midwest'\r\n'3' = ...
2,0,3.0,CENDIV,Char,1.0,$CENDIV.,Census division,'1' = 'New England'\r\n'2' = 'Middle Atlantic'...
3,0,4.0,PBA,Char,2.0,$PBA.,Principal building activity,'01' = 'Vacant'\r\n'02' = 'Office'\r\n'04' = '...


In [10]:
# NOTE: some codes span two rows
# need to find a way to concatenate this, then update map_codes function '(continued in next cell)' condition
category_dfs['1'].iloc[44:48]

,category,file_order,col_name,col_type,length,format,col_description,codes
48,1,49.0,ACT3PCT,Num,8.0,NaN,Percent used for third activity,1 - 100\r\nMissing = Not applicable
49,1,50.0,PBAPLUS,Char,2.0,$PBAPLUS.,More specific building activity,'01' = 'Vacant'\r\n'02' = 'Administrative/prof...
50,1,NaN,NaN,NaN,NaN,NaN,"More specific building activity, continued",'28' = 'Elementary/middle school'\r\n'29' = 'H...
51,1,51.0,DRYCL,Char,1.0,$DRYCLN.,Dry cleaning onsite,'1' = 'In this building'\r\n'2' = 'Off-site'\r...


In [11]:
# only category 10 has missing codes
category_dfs['10'].loc[:,'codes'] = category_dfs['10'].codes.fillna('no code')
category_dfs['10'].head()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,category,file_order,col_name,col_type,length,format,col_description,codes
1049,10,1049.0,HDD65,Num,8.0,NaN,Heating degree days (base 65),no code
1050,10,1050.0,CDD65,Num,8.0,NaN,Cooling degree days (base 65),no code
1051,10,1051.0,MFUSED,Char,1.0,$YESNO.,Any major fuel used,'1' = 'Yes'\r\n'2' = 'No'\r\nMissing = Not app...
1052,10,1052.0,MFBTU,Num,8.0,NaN,Annual major fuel consumption (thous Btu),no code
1053,10,1053.0,MFEXP,Num,8.0,NaN,Annual major fuel expenditures ($),no code


In [12]:
all_cats_code_maps = {}
for cat_num, df in category_dfs.items(): 
    print(f'category: {cat_num}')
    if cat_num != 'get rid of':
        i = 0
        category_code_df = category_dfs[cat_num].drop(['category', 'file_order', 'col_type', 'length', 'format',], axis=1)
        category_code_df['code_split'] = category_code_df.codes.str.split('\n')
        category_code_list = category_code_df.code_split.values
        category_cols = category_code_df.columns
        category_code_map = map_code_values(category_code_list, print_errors=True)
        for col_name in category_code_df.col_name.values:
            category_code_map[col_name] = category_code_map.pop(i)
            i += 1
        all_cats_code_maps[cat_num] = category_code_map

category: 0
category: 1
category: 2
category: 3
category: 4
category: 5
category: 6
category: 7
category: 10


In [21]:
for cat_num, code_map in all_cats_code_maps.items():
    print(cat_num)
    print(code_map)
    print('---------------------------------------------------------------------------------------------------------')

0
{'PUBID': {'00001 - 06720': 'continuous numerical range'}, 'REGION': {'1': 'Northeast', '2': 'Midwest', '3': 'South', '4': 'West'}, 'CENDIV': {'1': 'New England', '2': 'Middle Atlantic', '3': 'East North Central', '4': 'West North Central', '5': 'South Atlantic', '6': 'East South Central', '7': 'West South Central', '8': 'Mountain', '9': 'Pacific'}, 'PBA': {'01': 'Vacant', '02': 'Office', '04': 'Laboratory', '05': 'Nonrefrigerated warehouse', '06': 'Food sales', '07': 'Public order and safety', '08': 'Outpatient health care', '11': 'Refrigerated warehouse', '12': 'Religious worship', '13': 'Public assembly', '14': 'Education', '15': 'Food service', '16': 'Inpatient health care', '17': 'Nursing', '18': 'Lodging', '23': 'Strip shopping mall', '24': 'Enclosed mall', '25': 'Retail other than mall', '26': 'Service', '91': 'Other'}}
---------------------------------------------------------------------------------------------------------
1
{'FREESTN': {'1': 'Yes', 'Missing': 'No'}, 'SQFT': 

In [9]:
new_codes_df = pd.DataFrame(all_cats_code_maps)
print(new_codes_df.columns)
new_codes_df['0'].dropna()

Index(['0', '1', '2', '3', '4', '5', '6', '7', '10'], dtype='object')


PUBID       {'00001 - 06720': 'continuous numerical range'}
REGION    {'1': 'Northeast', '2': 'Midwest', '3': 'South...
CENDIV    {'1': 'New England', '2': 'Middle Atlantic', '...
PBA       {'01': 'Vacant', '02': 'Office', '04': 'Labora...
Name: 0, dtype: object

In [10]:
to_add_dfs = []
for category_num in new_codes_df.columns:
    to_add = new_codes_df[category_num].dropna().reset_index().rename(columns={'index':'col_name', category_num:'codes_dict'})
    to_add_dfs.append(to_add)
to_merge = pd.concat(to_add_dfs)

In [11]:
new_codebook = df_codebook.merge(to_merge, on='col_name')
new_codebook#.drop_duplicates(subset='col_name')

,category,file_order,col_name,col_type,length,format,col_description,codes,codes_dict
0,0,1.0,PUBID,Char,5.0,NaN,Building identifier,00001 - 06720,{'00001 - 06720': 'continuous numerical range'}
1,0,2.0,REGION,Char,1.0,$REGION.,Census region,'1' = 'Northeast'\n'2' = 'Midwest'\n'3' = 'Sou...,"{'1': 'Northeast', '2': 'Midwest', '3': 'South..."
2,0,3.0,CENDIV,Char,1.0,$CENDIV.,Census division,'1' = 'New England'\n'2' = 'Middle Atlantic'\n...,"{'1': 'New England', '2': 'Middle Atlantic', '..."
3,0,4.0,PBA,Char,2.0,$PBA.,Principal building activity,'01' = 'Vacant'\n'02' = 'Office'\n'04' = 'Labo...,"{'01': 'Vacant', '02': 'Office', '04': 'Labora..."
4,1,5.0,FREESTN,Char,1.0,$YES.,Freestanding building,1' = 'Yes'\nMissing='No',"{'1': 'Yes', 'Missing': 'No'}"
...,...,...,...,...,...,...,...,...,...
512,10,1115.0,DHCLBTU,Num,8.0,NaN,District heat cooling use (thous Btu),NaN,{'no code': 'continuous numerical value'}
513,10,1116.0,DHWTBTU,Num,8.0,NaN,District heat water heating use (thous Btu),NaN,{'no code': 'continuous numerical value'}
514,10,1117.0,DHCKBTU,Num,8.0,NaN,District heat cooking use (thous Btu),NaN,{'no code': 'continuous numerical value'}
515,10,1118.0,DHOTBTU,Num,8.0,NaN,District heat miscellaneous use (thous Btu),NaN,{'no code': 'continuous numerical value'}


In [12]:
new_codebook.to_csv('2012cbecs_codebook_wDicts.csv', index=False)